In [1]:
import torch

seq_len = 6
d_head = 6

Q = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
K = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
V = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10


In [2]:
Q

tensor([[0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000],
        [0.7000, 0.8000, 0.9000, 1.0000, 1.1000, 1.2000],
        [1.3000, 1.4000, 1.5000, 1.6000, 1.7000, 1.8000],
        [1.9000, 2.0000, 2.1000, 2.2000, 2.3000, 2.4000],
        [2.5000, 2.6000, 2.7000, 2.8000, 2.9000, 3.0000],
        [3.1000, 3.2000, 3.3000, 3.4000, 3.5000, 3.6000]])

In [3]:
torch.softmax(Q @ K.T, dim=1) @ V

tensor([[2.8643, 2.9643, 3.0643, 3.1643, 3.2643, 3.3643],
        [3.0797, 3.1797, 3.2797, 3.3797, 3.4797, 3.5797],
        [3.0977, 3.1977, 3.2977, 3.3977, 3.4977, 3.5977],
        [3.0997, 3.1997, 3.2997, 3.3997, 3.4997, 3.5997],
        [3.1000, 3.2000, 3.3000, 3.4000, 3.5000, 3.6000],
        [3.1000, 3.2000, 3.3000, 3.4000, 3.5000, 3.6000]])

In [4]:
torch.softmax(Q[0, :] @ K.T, dim=0)

tensor([0.0013, 0.0046, 0.0164, 0.0577, 0.2033, 0.7167])

# One-shot

In [6]:
seq_len = 4
d_head = 4

Q = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
K = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
V = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
O = torch.zeros((seq_len, d_head))


m = torch.full((seq_len, 1), -torch.inf)
l = torch.zeros((seq_len, 1))

prev_rowmax = m  # shape Br x 1
prev_denominator = l  # shape Br x 1

# print("====== Q =======")
# print(Q)
# print()

# print("====== Sij =======")
Sij = Q @ K # TODO: I removed the transpose here to match GPU version 
# print(Sij)
# print()

# print("====== tile_rowmax =======")
tile_rowmax = torch.max(Sij, dim=1).values[:, None]
# print(tile_rowmax)
# print()

print("====== tile_numerator =======")
tile_numerator = torch.exp(Sij - tile_rowmax)
print(tile_numerator)
print()

print("====== tile_denominator =======")
tile_denominator = torch.sum(tile_numerator, dim=1)[:, None]
print(tile_denominator)
print()

print("====== new_rowmax =======")
new_rowmax = torch.max(torch.column_stack([prev_rowmax, tile_rowmax]), dim=1).values[:, None]
print(new_rowmax)
print()

print("====== update_prev_exponent  =======")
update_prev_exponent = torch.exp(prev_rowmax - new_rowmax)
print(update_prev_exponent)

print("====== new_denominator =======")
new_denominator = prev_denominator * update_prev_exponent + torch.exp(tile_rowmax - new_rowmax) * tile_denominator
print(new_denominator)
print()

# print("====== O =======")
# O = (li * torch.exp(mi - mi_new) * O / li_new) + (torch.exp(tile_rowmax - mi_new) * pij_hat / li_new) @ V
# print(O)
# print()


====== tile_numerator =======
tensor([[0.7408, 0.8187, 0.9048, 1.0000],
        [0.4584, 0.5945, 0.7711, 1.0000],
        [0.2837, 0.4317, 0.6570, 1.0000],
        [0.1755, 0.3135, 0.5599, 1.0000]])

====== tile_denominator =======
tensor([[3.4644],
        [2.8240],
        [2.3724],
        [2.0489]])

====== new_rowmax =======
tensor([[1.2000],
        [2.8000],
        [4.4000],
        [6.0000]])

====== update_prev_exponent  =======
tensor([[0.],
        [0.],
        [0.],
        [0.]])
====== new_denominator =======
tensor([[3.4644],
        [2.8240],
        [2.3724],
        [2.0489]])



In [20]:
import numpy as np
np.exp(-float("inf"))

0.0

# 2-shot on K

In [6]:
seq_len = 4
d_head = 4

Q = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
K = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
V = (torch.arange(seq_len * d_head).reshape(seq_len, d_head) + 1.)  / 10
O = torch.zeros((seq_len, d_head))


m = torch.full((seq_len, 1), -torch.inf)
l = torch.zeros((seq_len, 1))

mi = m  # shape Br x 1
li = l  # shape Br x 1

# print("1st pass on K")
print("====== Q =======")
print(Q)
print()

print("====== Sij =======")
Sij = Q @ K[0:2, :].T
print(Sij)
print()

print("====== mij_hat =======")
mij_hat = torch.max(Sij, dim=1).values[:, None]
print(mij_hat)
print()

print("====== pij_hat =======")
pij_hat = torch.exp(Sij - mij_hat)
print(pij_hat)
print()

print("====== lij_hat =======")
lij_hat = torch.sum(pij_hat, dim=1)[:, None]
print(lij_hat)
print()

print("====== mi_new =======")
mi_new = torch.max(torch.column_stack([mi, mij_hat]), dim=1).values[:, None]
print(mi_new)
print()

print("====== li_new =======")
li_new = torch.exp(mi - mi_new) * li + torch.exp(mij_hat - mi_new) * lij_hat
print(li_new)
print()

print("====== O =======")
O = (li * torch.exp(mi - mi_new) * O / li_new) + (torch.exp(mij_hat - mi_new) * pij_hat / li_new) @ V[0:2, :]
print(O)
print()

mi = mi_new


====== Q =======
tensor([[0.1000, 0.2000, 0.3000, 0.4000],
        [0.5000, 0.6000, 0.7000, 0.8000],
        [0.9000, 1.0000, 1.1000, 1.2000],
        [1.3000, 1.4000, 1.5000, 1.6000]])

====== Sij =======
tensor([[0.3000, 0.7000],
        [0.7000, 1.7400],
        [1.1000, 2.7800],
        [1.5000, 3.8200]])

====== mij_hat =======
tensor([[0.7000],
        [1.7400],
        [2.7800],
        [3.8200]])

====== pij_hat =======
tensor([[0.6703, 1.0000],
        [0.3535, 1.0000],
        [0.1864, 1.0000],
        [0.0983, 1.0000]])

====== lij_hat =======
tensor([[1.6703],
        [1.3535],
        [1.1864],
        [1.0983]])

====== mi_new =======
tensor([[0.7000],
        [1.7400],
        [2.7800],
        [3.8200]])

====== li_new =======
tensor([[1.6703],
        [1.3535],
        [1.1864],
        [1.0983]])

====== O =======
tensor([[0.3395, 0.4395, 0.5395, 0.6395],
        [0.3955, 0.4955, 0.5955, 0.6955],
        [0.4372, 0.5372, 0.6372, 0.7372],
        [0.4642, 0.5642, 0.664

In [7]:
# print("2nd pass on K")
print("====== Q =======")
print(Q)
print()

print("====== Sij =======")
Sij = Q @ K[2:4, :].T
print(Sij)
print()

print("====== mij_hat =======")
mij_hat = torch.max(Sij, dim=1).values[:, None]
print(mij_hat)
print()

print("====== pij_hat =======")
pij_hat = torch.exp(Sij - mij_hat)
print(pij_hat)
print()

print("====== lij_hat =======")
lij_hat = torch.sum(pij_hat, dim=1)[:, None]
print(lij_hat)
print()

print("====== mi_new =======")
mi_new = torch.max(torch.column_stack([mi, mij_hat]), dim=1).values[:, None]
print(mi_new)
print()

print("====== li_new =======")
li_new = torch.exp(mi - mi_new) * li + torch.exp(mij_hat - mi_new) * lij_hat
print(li_new)
print()

print("====== O =======")
O = (li * torch.exp(mi - mi_new) * O / li_new) + (torch.exp(mij_hat - mi_new) * pij_hat / li_new) @ V[2:4, :]
print(O)
print()

====== Q =======
tensor([[0.1000, 0.2000, 0.3000, 0.4000],
        [0.5000, 0.6000, 0.7000, 0.8000],
        [0.9000, 1.0000, 1.1000, 1.2000],
        [1.3000, 1.4000, 1.5000, 1.6000]])

====== Sij =======
tensor([[1.1000, 1.5000],
        [2.7800, 3.8200],
        [4.4600, 6.1400],
        [6.1400, 8.4600]])

====== mij_hat =======
tensor([[1.5000],
        [3.8200],
        [6.1400],
        [8.4600]])

====== pij_hat =======
tensor([[0.6703, 1.0000],
        [0.3535, 1.0000],
        [0.1864, 1.0000],
        [0.0983, 1.0000]])

====== lij_hat =======
tensor([[1.6703],
        [1.3535],
        [1.1864],
        [1.0983]])

====== mi_new =======
tensor([[1.5000],
        [3.8200],
        [6.1400],
        [8.4600]])

====== li_new =======
tensor([[1.6703],
        [1.3535],
        [1.1864],
        [1.0983]])

====== O =======
tensor([[1.1395, 1.2395, 1.3395, 1.4395],
        [1.1955, 1.2955, 1.3955, 1.4955],
        [1.2372, 1.3372, 1.4372, 1.5372],
        [1.2642, 1.3642, 1.464

In [8]:
print(torch.exp(mi - mi_new) * li)
print(torch.exp(mij_hat - mi_new) * lij_hat)
print(li_new)

tensor([[0.],
        [0.],
        [0.],
        [0.]])
tensor([[1.6703],
        [1.3535],
        [1.1864],
        [1.0983]])
tensor([[1.6703],
        [1.3535],
        [1.1864],
        [1.0983]])


In [9]:
1.6703 + 0.7505

2.4208

In [10]:
tensor([[0.8915, 0.9915, 1.0915, 1.1915],
        [1.1067, 1.2067, 1.3067, 1.4067],
        [1.2103, 1.3103, 1.4103, 1.5103],
        [1.2566, 1.3566, 1.4566, 1.5566]])

NameError: name 'tensor' is not defined

# Softmax

> https://arxiv.org/pdf/1805.02867.pdf

In [ ]:
torch.manual_seed(0)

x = torch.arange(10, dtype=torch.float32)
x

In [ ]:
def naive_softmax(vec):
   return torch.exp(vec) / torch.sum(torch.exp(vec))

torch.allclose(torch.softmax(x.clone(), dim=0), naive_softmax(x.clone()))

In [ ]:
def safe_softmax(vec):
    # Compute row max in 1st pass
    row_max = torch.max(vec)
    # Compute denominator in 2nd pass
    denominator = torch.sum(torch.exp(vec - row_max))
    return torch.exp(vec - row_max) / denominator


torch.allclose(torch.softmax(x.clone(), dim=0), safe_softmax(x.clone()))

In [ ]:
#TODO: Cf goodnotes ipad "Flash attention   "

def online_softmax(vec):
    N = len(vec)
    rowmax = -float("inf")
    denominator = 0
    # Compute the max and denominator in single pass
    for i in range(N):
        prev_rowmax = rowmax
        rowmax = max(prev_rowmax, vec[i])
        denominator = denominator * torch.exp(prev_rowmax - rowmax) + torch.exp(vec[i] - rowmax)
        print(f"prev_rowmax = {prev_rowmax} | rowmax = {rowmax} | denominator = {denominator}")

    print(torch.exp(vec - rowmax))
    print(denominator)
    return torch.exp(vec - rowmax) / denominator

torch.allclose(torch.softmax(x.clone(), dim=0), online_softmax(x.clone()))

In [ ]:
print(x)

In [ ]:
online_softmax(x.clone())

In [ ]:
torch.exp(x - torch.max(x))

In [ ]:
torch.sum(torch.exp(x - torch.max(x)))

In [ ]:
torch.exp(x - torch.max(x)) / torch.sum(torch.exp(x - torch.max(x)))